# 🧵 Multithreading in Python

## ❓ Can Python do multithreading?

The short answer: **yes... but** it really depends on what we mean by "multithreading". First, let's clarify some concepts.

## 📦 What Is a Process?

A **process** is an instance of a running program.

When you launch an application (a browser, a Python script, a text editor), the operating system creates a process for it. Each process has:

* Its own memory space
* Its own variables and data
* Its own resources (file handles, network connections, etc.)

Processes are **isolated** from each other by the operating system. One process cannot directly access another process's memory. This isolation improves stability and security but comes with a cost: communication between processes is relatively slow.

To list all running processes on your system, you can use commands like `ps` (Linux/Mac) or use `Task Manager` (Windows).

## 🧶 What Is a Thread?

A **thread** is a smaller unit of execution *inside* a process.

A single process can contain:

* One thread (single-threaded process)
* Multiple threads (multi-threaded process)

Threads within the same process:

* Share the same memory space
* Share variables and data
* Can communicate very efficiently

However, because they share memory, threads must be carefully coordinated to avoid race conditions and data corruption.

An exemple of process could be a chrome tab, and the process it runs could be represented as:

```
Process: Chrome tab
├─ UI thread
├─ JavaScript engine thread
├─ Network thread
├─ Rendering thread
```

And other example could be a text editor:

```
Process: Text Editor
├─ Main thread → UI event loop
├─ Auto-save thread
├─ Spell-check thread
```

## 🎯 Applications : I/O-bound vs CPU-bound

* **I/O-bound applications**: spend most of their time waiting for input/output operations (disk, network, user input). Examples: web servers, file processing, database queries.
  
* **CPU-bound applications**: spend most of their time performing computations. Examples: scientific calculations, image processing, data analysis.
 
## ⚖️ Multiprocessing vs Multithreading

### 🔀 Multiprocessing

**Multiprocessing** means running multiple processes in parallel.

Characteristics:

* Separate memory spaces
* Higher memory usage
* More expensive communication
* Strong isolation and safety

Typical use cases:

* CPU-bound tasks
* Parallel computation
* Workloads that must scale across CPU cores

In the case of Python doing some multiprocessing means having several independent Python interpreters running simultaneously, each in its own process.
  
### 🧵 Multithreading

**Multithreading** means running multiple threads within the same process.

Characteristics:

* Shared memory
* Low communication overhead
* Faster context switching
* Higher risk of concurrency bugs

Typical use cases:

* I/O-bound tasks (waiting for files, network, databases)
* Responsive applications (GUIs, servers)

In the case of Python doing some multithreading means having several threads running inside the same Python interpreter process.

### 🔑 Key Conceptual Difference

| Aspect          | Multithreading    | Multiprocessing |
| --------------- | ----------------- | --------------- |
| Memory          | Shared            | Isolated        |
| Communication   | Fast              | Slower          |
| Safety          | Risk of races     | Safer           |
| CPU parallelism | Limited in Python | Full            |

## 💻 CPU Cores and Hardware Parallelism

A **CPU core** is a physical execution unit inside a processor capable of executing instructions independently.

Modern CPUs typically contain:

* Multiple physical cores
* Sometimes multiple *hardware threads* per core (e.g., Hyper-Threading)

Each core can execute **one instruction stream at a time**. With multiple cores, a CPU can truly run multiple tasks simultaneously.

## 📅 A Short History: Before and After 2005

Before roughly **2005**, most consumer processors had:

* A single core
* Increasing performance achieved by higher clock speeds

Parallelism was mostly an operating-system illusion created through **time slicing**: rapidly switching between tasks.

Around 2005, physical limits (heat, power consumption) stopped clock speeds from increasing significantly. The industry shifted toward:

* Multi-core processors
* Hardware-level parallelism

Since then, software has increasingly needed to be written with **concurrency and parallelism in mind**.

## 🔄 Concurrency vs Parallelism

These terms are often confused but are not the same.

* **Concurrency**: multiple tasks *making progress* during the same time period
* **Parallelism**: multiple tasks executing *at the same time* on different cores

A single-core CPU can support concurrency.
A multi-core CPU enables true parallelism.

## 🔒 The Global Interpreter Lock (GIL)

### 🤔 What Is the GIL?

When Python was created in the late 1980s, CPU architectures were mostly single-core. To simplify memory management and ensure thread safety, the creators of Python introduced the **Global Interpreter Lock (GIL)** in 1992.

The GIL prevents multiple native threads from executing Python bytecodes at once. This means that even if you have multiple threads, only one can run Python code at any given moment.

At first glance, this seems to appear as a major limitation of the language, especially when compared to other programming languages that support multithreading more natively (C++, Rust, Java, etc.) and the GIL has been a topic of much debate in the Python community.

However, as Larry Hastings, a core Python developer, explained, the GIL is probably one of the reasons why Python has become so popular.

The **Global Interpreter Lock (GIL)** is a mutex (mutual exclusion lock) and exists only in **CPython**, the reference implementation of Python.

It enforces a rule:

> Only **one thread** may execute Python bytecode at a time *per process*.

This means that even on a multi-core machine, Python threads cannot execute Python code in parallel within the same process.

<img src='files/gil.png' alt='GIL diagram' width='600'>

### ❓ Why Does the GIL Exist?

The GIL simplifies:

* Memory management
* Garbage collection
* Thread safety of Python objects

As a result, many single-threaded Python programs are **faster and simpler** than they would be without the GIL.

### ✅ What the GIL Does *Not* Prevent

* Threads can still run concurrently
* Threads can release the GIL during blocking I/O
* Native extensions written in C can release the GIL

This is why multithreading in Python is still very useful for:

* I/O-bound workloads
* Network servers
* Waiting-heavy tasks

### 📚 References about the GIL

* ["It isn't Easy to Remove the GIL" by Guido van van Rossum (2007)](https://www.artima.com/weblogs/viewpost.jsp?thread=214235)
* [Larry Hastings on the GIL at PyCon (2015)](https://www.youtube.com/watch?v=KVKufdTphKs)
* [Guido van Rossum interview on the GIL (2022)](https://www.youtube.com/watch?v=m4zDBk0zAUY)


### 🎯 So is Python multithreaded or not?

**>>>** Technically Python is multithreaded but it is not "simultaneously multithreaded".

## 📦 The threading Module

Python provides a built-in `threading` module to create and manage threads.

In [ ]:
import threading
import time

### 🧩 The object `Thread`

In Python, a thread is represented by the `Thread` class in the `threading` module. You can create a new thread by instantiating this class and passing a target function to run in that thread.

In [ ]:
# A function to be run in a thread
def print_number():
    for i in range(5):
        print(f"Hello from the thread! number is {i}")
        time.sleep(0.5)

# Main thread execution
thread = threading.Thread(target=print_number)
thread.start()
print("Hello from the main thread!")

As you can see with the example above, the `Thread` class allows you to create a thread that runs a specific function concurrently with the main thread.

But something went wrong and the print from the main thread was printed before the new thread was finished.

Let's fix this.

### 🔗 Using the .`join()` Method

The `join()` method allows the main thread to wait for a specific thread to finish before continuing. This ensures that the main thread does not exit before the new thread has completed its execution.

In [ ]:
# A function to be run in a thread
def print_number():
    for i in range(5):
        print(f"Hello from the thread! number is {i}")
        time.sleep(0.5)

# Main thread execution
thread = threading.Thread(target=print_number)
thread.start()
thread.join()  # Wait for the thread to finish
print("Hello from the main thread!")

### 📥 Using threads with arguments

In [ ]:
# A function to be run in a thread
def print_number(message, count):
    for i in range(count):
        print(f"{message} number is {i}")
        time.sleep(0.5)

# Main thread execution
thread = threading.Thread(target=print_number,
                          args=("Hello from the thread!", 5))
thread.start()
thread.join()  # Wait for the thread to finish
print("Hello from the main thread!")

### 🔢 Creating several threads at once

In [ ]:
# A function to be run in a thread
def print_number(message, count):
    for i in range(count):
        print(f"{message} number is {i}")
        time.sleep(0.5)

# Main thread execution
threads = []
for i in range(3):
    thread = threading.Thread(target=print_number,
                              args=(f"{' ' * i}Thread-{i+1}", 5))
    threads.append(thread)
    thread.start()
    # thread.join()  # this would make threads run sequentially

for thread in threads:
    thread.join()  # Wait for all threads to finish
print("Hello from the main thread!")

### 👻 Dameon Threads

A daemon thread is a thread that runs in the background and does not prevent the program from exiting. When the main program exits, all daemon threads are terminated automatically.

Cautious ! The following example works inside a .py but not inside a Jupyter notebook because the kernel doesn't die after you run a cell so the daemon is still working in the background.

### ⚠️ Race Conditions

Here using the expression time.sleep(0) releases the GIL and tells the scheduler: "switch threads if you want".


In [ ]:
import threading
import time

counter = 0

def increment():
    global counter
    for _ in range(10_000):
        tmp = counter
        time.sleep(0)  # force a context switch
        counter = tmp + 1

threads = [threading.Thread(target=increment) for _ in range(4)]

for t in threads:
    t.start()
for t in threads:
    t.join()

print(counter)

The output should be 40 000 but instead we get 10 000 or 10 001 or 10 002...

Why? Because all threads are accessing and modifying the shared variable `counter` simultaneously without any synchronization mechanism. So most of the increments are lost. But sometimes a few increments succeed before another thread overwrites the value.

So Why the GIL doesn’t save us here? Because the doesn't prevent race conditions. The GIL only ensures that only one thread executes Python bytecode at a time, but your race happens between bytecode instructions.

### 🔐 Using `threading.Lock()`

To fix the race condition, we can use a `Lock` from the `threading` module. A lock is a synchronization primitive that can be used to ensure that only one thread can access a shared resource at a time.

In [ ]:
import threading
import time

counter = 0
counter_lock = threading.Lock() # create a lock object

def increment():
    global counter
    with counter_lock: # acquire the lock
        for _ in range(10_000):
            tmp = counter
            time.sleep(0)
            counter = tmp + 1

threads = [threading.Thread(target=increment) for _ in range(4)]

for t in threads:
    t.start()
for t in threads:
    t.join()

print(counter)

### 🎯 The module Concurrent.futures

The `concurrent.futures` module provides a high-level interface for asynchronously executing callables.
It provides two main classes:
* `ThreadPoolExecutor`: for managing a pool of threads
* `ProcessPoolExecutor`: for managing a pool of processes

#### 🤔 What is an executor?

An **executor** is an object that manages a pool of threads or processes and provides methods to submit tasks for execution.

It abstracts away the low-level details of thread or process management, allowing you to focus on defining the tasks you want to run concurrently.


#### ⚖️ What is the difference between ThreadPoolExecutor and ProcessPoolExecutor?

* `ThreadPoolExecutor` is used for I/O-bound tasks where threads can be beneficial despite the GIL, as they can release the GIL during blocking operations.
  
* `ProcessPoolExecutor` is used for CPU-bound tasks where true parallelism is needed, as each process has its own Python interpreter and memory space, bypassing the GIL. More about this in the next chapter.

In [ ]:
# With ThreadPoolExecutor

import threading
import concurrent.futures
import time

def task(n):
    print(f"{" " * n}Task {n} starting on thread {threading.current_thread().name}")
    time.sleep(n)
    print(f" {" " * n}Task {n} completed on thread {threading.current_thread().name}")
    return n * n

with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    results = executor.map(task, [1, 2, 3, 4, 5])

print("Results:", list(results))
print("Main thread finished.")

Looking at the outputs, you can see  the executor decided on its own which task to run on which thread or process.

### 🚦 Thread Events

A `threading.Event` is a synchronization primitive that allows threads to communicate with each other by signaling events. An event can be in one of two states: "set" or "clear". Threads can wait for an event to be set before proceeding with their execution.

So let's say we have a thread and we want to start at a very specific moment.

In [ ]:
def worker(event):
    print("    Worker is waiting for the event to be set.")
    event.wait()  # Wait until the event is set
    print("    Worker has detected the event is set and is proceeding.")

    for _ in range(5):
        print("    Worker is working...")
        time.sleep(1)
    print("    Worker has finished its work.")
    
event = threading.Event()
thread = threading.Thread(target=worker, args=(event,))
thread.start()
time.sleep(3)  # Simulate some setup time in the main thread
print("Main thread is setting the event.")
event.set()  # Signal the event
thread.join()
print("Main thread finished.")

### 💪 Exercice

The following code fetches data from various websites. Modify the function `run_fetch_url` and use multithreading to fetch the data concurrently, improving the overall execution time.

You can either:

- Use `threading` to create and manage threads manually.
- Use `concurrent.futures.ThreadPoolExecutor` for a higher-level approach.
- (Do both!)

In [ ]:
import requests
import time

def fetch_url(url):
    response = requests.get(url)
    print(f"""Fetched {url} with status code {response.status_code}.
            Data is {len(response.content)} bytes.""")

def run_fetch_url(urls):
    start_time = time.time()
    for url in urls:fetch_url(url)
    end_time = time.time()
    print(f"Total time taken: {end_time - start_time} seconds")

urls = ['https://stackoverflow.com',
        'https://github.com',
        'https://python.org',
        'https://wikipedia.org',
        'https://reddit.com',
        'https://google.com',
        'https://youtube.com',]

run_fetch_url(urls)

In [ ]:
# Code the new function here!

